##### Welcome

In [11]:
print("-"*40)
print("Welcome to the image convolver thing!\nPlease ensure that the image is present in the same directory as this program, and that the following dependencies are installed:\n\t- NumPy\n\t- Pillow\n\t- PyTorch\n")

----------------------------------------
Welcome to the image convolver thing!
Please ensure that the image is present in the same directory as this program, and that the following dependencies are installed:
	- NumPy
	- Pillow
	- PyTorch



##### Import Packages

In [12]:
print("Checking dependencies...")
pos = 0
dependency_list = ["Pillow", "NumPy", "PyTorch"]
try:
    from PIL import Image
    pos +=1
    import numpy as np
    pos +=1
    import torch
    from torch import nn
    pos +=1
    import sys
except:
    sys.exit(f"Dependency error for {dependency_list[pos]}")

print("Dependency checks complete.")

Checking dependencies...
Dependency checks complete.


In [13]:
debug_mode = 1

if (debug_mode):
    image_name = "testfile.jpg"
else:
    image_name = input("Enter the name of the file (including the file extension, e.g. example.png): ")

'''
Goals: 
1. mean blur
2. gaussian blur
3. bokeh filter
3. edge detection: sobel + canny
4. foreground/background blur and detection
'''

'\nGoals: \n1. mean blur\n2. gaussian blur\n3. bokeh filter\n3. edge detection: sobel + canny\n4. foreground/background blur and detection\n'

##### Open image

In [14]:
image_file = Image.open(f"./{image_name}")
image_array = np.asarray(image_file)
print(image_array.shape)
# i = Image.fromarray(image_array)
# i.save("copy.jpg")

(720, 1280, 3)


##### Define Cross Correlation, i.e. Convolution function

In [15]:
def corr2d(X, K):
    """
    Compute single channel 2D cross-correlation.
    X: Input array, i.e. channel
    K: Kernel
    Y: Output array, i.e. feature map
    Credit: Dive into Deep Learning, A. Zhang et al
    """

    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    
    return Y

In [16]:
def corr2d_separable(X, k):
    '''
    Compute single channel 2D cross-correlation, but significantly faster by using separable kernels.
    Performs two linear passes with the kernel vector, one with the linear kernel, followed by one with its transpose.
    X: Input array, i.e. channel
    k: Kernel vector
    Y: Output array, i.e. feature map
    '''
    k = k.reshape(-1,1)
    first_pass_featuremap = corr2d(X,k)
    Y = corr2d(first_pass_featuremap,k.T)
    
    return second_pass_featuremap  

In [17]:
def corr2d_separable_multichannel(X, k):
    '''
    Compute multichannel channel 2D cross-correlation using separable kernels.
    X: Input array, i.e. channels
    k: Kernel vector
    Y: Output array, i.e. feature map
    '''
    # Future: Implement true multichannel support, where num_channels is an input
    R = X[:,:,0]
    G = X[:,:,1]
    B = X[:,:,2]
    k = k.reshape(-1,1)
    
    first_pass_featuremap_R = corr2d(R,k)
    final_featuremap_R = corr2d(first_pass_featuremap_R,k.T)

    first_pass_featuremap_G = corr2d(G,k)
    final_featuremap_G = corr2d(first_pass_featuremap_G,k.T)

    first_pass_featuremap_B = corr2d(B,k)
    final_featuremap_B = corr2d(first_pass_featuremap_B,k.T)

    Y = np.append(final_featuremap_R, np.append(final_featuremap_G, final_featuremap_B, axis = 2), axis=2)
    
    return Y  

##### Define filters

In [18]:
def mean_blur(image_array, kernel_size=3, save=False):
    '''
    Implements mean blur. 
    Essentially, each new pixel is just the mean of the color values 
    '''

    kernel = np.ones(kernel_size)
    
    featuremap = corr2d_separable_multichannel(image_array, kernel)
    featuremap = featuremap/(kernel_size**2)

    mean_blur_image = Image.fromarray(featuremap)
    mean_blur_image.show()
    if (save):
        mean_blur_image.save(f"{image_name}_mean_blur.jpg")

    return featuremap

In [19]:
def generate_kernel_gaussian_separable(kernel_size=5):
    if (kernel_size%2==0):
        mid_val = kernel_size/2
    else:
        mid_val = int(kernel_size/2) + 1

    kernel = np.arange(1, mid_val+1)
    kernel = np.append(kernel, kernel[:-1])
        
    return kernel

print(generate_kernel_gaussian_separable())

[1 2 3 1 2]


In [20]:
mean_blur(image_array=image_array, kernel_size=40)